# Overview
Although typically referred to by a single name, Bokeh comprises two separate libraries.

The first component is a JavaScript library, BokehJS, that runs in the browser. This library is responsible for all of the rendering and user interaction. Its input is a collection of declarative JSON objects that comprise a “scenegraph”. The objects in this scenegraph describe everything that BokehJS should handle: what plots and widgets are present and in what arrangement, what tools and renderers and axes the plots will have, etc. These JSON objects are converted into JavaScript objects in the browser.

The second component is a library in Python (or other languages) that can generate the JSON described above. In the Python Bokeh library, this is accomplished at the lowest level by exposing a set of Model subclasses that exactly mirror the set of BokehJS Models that are use in the browser. Most of the models are very simple, usually consisting of a few property attributes and no methods. Model attributes can either be configured when the model is created, or later by setting attribute values on the model object:

##### properties can be configured when a model object is initialized
glyph = Rect(x="x", y="y2", w=10, h=20, line_color=None)

##### properties can be configured by assigning values to attributes on exitsting models
glyph.fill_alpha = 0.5
glyph.fill_color = "navy"

These methods of configuration work in general for all Bokeh models.

Bokeh models are eventually collected into a Bokeh Document for serialization between BokehJS and the Bokeh Python bindings. As an example, the following image shows an example document for simple plot:

Such a document could be created at a high level using the bokeh.plotting API, which automatically assembles any models such as axes and grids and toolbars in a reasonable way. However it is also possible to assemble all the components "by hand" using the low-level bokeh.models API. Using the bokeh.models interface provides complete control over how Bokeh plots and Bokeh widgets are put together and configured. However, it provides no help with assembling the models in meaningful or correct ways. It is entirely up to developers to build the scenegraph “by hand”.

# Example Walkthrough
Let's try to reproduce this NYTimes interactive chart Usain Bolt vs. 116 years of Olympic sprinters using the bokeh.models interface.

The first thing we need is to get the data. The data for this chart is located in the bokeh.sampledata module as a Pandas DataFrame. You can see the first ten rows below:

In [2]:
from bokeh.sampledata.sprint import sprint


In [3]:
sprint[:10]

,Name,Country,Medal,Time,Year
0,Usain Bolt,JAM,GOLD,9.63,2012
1,Yohan Blake,JAM,SILVER,9.75,2012
2,Justin Gatlin,USA,BRONZE,9.79,2012
3,Usain Bolt,JAM,GOLD,9.69,2008
4,Richard Thompson,TRI,SILVER,9.89,2008
5,Walter Dix,USA,BRONZE,9.91,2008
6,Justin Gatlin,USA,GOLD,9.85,2004
7,Francis Obikwelu,POR,SILVER,9.86,2004
8,Maurice Greene,USA,BRONZE,9.87,2004
9,Maurice Greene,USA,GOLD,9.87,2000


In [4]:
sprint.head()

,Name,Country,Medal,Time,Year
0,Usain Bolt,JAM,GOLD,9.63,2012
1,Yohan Blake,JAM,SILVER,9.75,2012
2,Justin Gatlin,USA,BRONZE,9.79,2012
3,Usain Bolt,JAM,GOLD,9.69,2008
4,Richard Thompson,TRI,SILVER,9.89,2008


In [8]:
import pandas as pd

In [17]:
sprint.groupby(['Year','Medal','Country']).mean().tail(3)

Time
Year Medal  Country      
2012 BRONZE USA      9.79
     GOLD   JAM      9.63
     SILVER JAM      9.75

Next we import some of the Bokeh models that need to be assembled to make a plot. At a minimum, we need to start with Plot, the glyphs (Circle and Text) we want to display, as well as ColumnDataSource to hold the data and range obejcts to set the plot bounds.

In [5]:
from bokeh.io import output_notebook, show
from bokeh.models.glyphs import Circle, Text
from bokeh.models import ColumnDataSource, Range1d, DataRange1d, Plot

In [6]:
output_notebook()

Loading BokehJS ...

# Setting up Data
Next we need set up all the columns we want in our column data source. Here we add a few extra columns like MetersBack and SelectedName that we will use for a HoverTool later.

In [18]:
abbrev_to_country = {
    "USA": "United States",
    "GBR": "Britain",
    "JAM": "Jamaica",
    "CAN": "Canada",
    "TRI": "Trinidad and Tobago",
    "AUS": "Australia",
    "GER": "Germany",
    "CUB": "Cuba",
    "NAM": "Namibia",
    "URS": "Soviet Union",
    "BAR": "Barbados",
    "BUL": "Bulgaria",
    "HUN": "Hungary",
    "NED": "Netherlands",
    "NZL": "New Zealand",
    "PAN": "Panama",
    "POR": "Portugal",
    "RSA": "South Africa",
    "EUA": "United Team of Germany",
}

fill_color = { "gold": "#efcf6d", "silver": "#cccccc", "bronze": "#c59e8a" }
line_color = { "gold": "#c8a850", "silver": "#b0b0b1", "bronze": "#98715d" }

def selected_name(name,medal,year):
    return name if medal=='gold' and year in [1988,1968,1936,1896] else ''

t0=sprint.Time[0]

In [19]:
sprint["Abbrev"]       = sprint.Country

In [21]:
sprint["Country"]=sprint.Abbrev.map(lambda abbr: abbrev_to_country[abbr])

In [23]:
sprint['Medal']=sprint.Medal.map(lambda medal:medal.lower())
sprint['Speed']=100.0/sprint.Time